# Retail Data Discovery and Processing

## Background 

This notebook was published as part of an AWS ML blog: TODO LINK. It contains data discovery and processing steps to prepare a dataset that is used in this <a href= "https://github.com/aws-samples/aws-sagemaker-ml-blog-predictive-campaigns/blob/master/notebooks/LSTM%2BPurchase%2BPredictor.ipynb"> notebook </a>

The dataset referenced by this notebook originates from the public UCI Machine Learning Repository: http://archive.ics.uci.edu/ml/datasets/online+retail

<b> Source: </b>

Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.

<b> Data Set Information: </b>

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

<b> Attribute Information: </b>

InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. Description: Product (item) name. Nominal. Quantity: The quantities of each product (item) per transaction. Numeric.
InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated. UnitPrice: Unit price. Numeric, Product price per unit in sterling. CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. Country: Country name. Nominal, the name of the country where each customer resides.

# Setup

This notebook was created and tested on an <i> ml.t2.medium </i> notebook instance running on the Sparkmagic (PySpark3) kernel, and an external multi-node EMR cluster. Follow the documention <a href= "https://github.com/aws/sagemaker-spark/tree/master/sagemaker-pyspark-sdk#running-on-sagemaker-notebook-instances"> to have your Sagemaker notebook instance connect to EMR.

Begin by...
1. Downloading the <a href= "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"> dataset </a>.
2. Upload the data set to an S3 bucket.
3. Start up a notebook instance and ensure the notebook instance has access to the data set. 

Import the required dependencies.

In [2]:
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf, lit
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from time import mktime, strptime

Set S3_BUCKET to the name of your S3 BUCKET. Set the S3_TARGET_PREFIX to the location of your dataset

In [3]:
S3_BUCKET = "dtong-ml-datasets"
S3_TARGET_PREFIX = "/raw/ecommerce-data.csv.gz"
S3_LOCATION = "s3://"+S3_BUCKET+S3_TARGET_PREFIX 
print(S3_LOCATION)

s3://dtong-ml-datasets/raw/ecommerce-data.csv.gz

Load the dataset into a Spark dataframe.

In [4]:
dateToTsUdf = udf(lambda date: int(mktime(strptime(date,"%m/%d/%Y %H:%M"))) if date is not None else None)

invoiceSchema =  StructType([StructField('InvoiceNo', IntegerType(), False),
                    StructField('StockCode', StringType(), False),
                    StructField('Description', StringType(), True),
                    StructField('Quantity', IntegerType(), False),
                    StructField('InvoiceDate', StringType(), False),
                    StructField('Price', FloatType(), False),
                    StructField('CustomerID', IntegerType(), True),
                    StructField('Country', StringType(), False)])

invoicesDf= spark.read.options(header=True).schema(invoiceSchema).csv(S3_LOCATION) 
invoicesDf=invoicesDf.withColumn("InvoiceDate", dateToTsUdf(invoicesDf.InvoiceDate).cast(IntegerType()))

invoicesDf.show(2)
invoicesDf.printSchema()

+---------+---------+--------------------+--------+-----------+-----+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|Price|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+-----+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 1291191960| 2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 1291191960| 3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+-----+----------+--------------+
only showing top 2 rows

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

The data set contains dirty data that we want to include in our training and test sets. This include transcations related to refunds, discounts, postage and such. After cleansing, the data set contains 396,485 transcations.

In [5]:
invoicesDf.registerTempTable("invoices")
invoicesCleanDf = sqlContext.sql(
    "select * \
    from invoices \
    where Description is not null \
    and Description != 'Manual' \
    and Description != 'POSTAGE' \
    and Description != 'Discount' \
    and CustomerID is not null \
    and Quantity > 0 \
    and Price > 0 \
    and Description != 'DOTCOM POSTAGE' \
    and Description != 'AMAZON FEE'"
)
invoicesCleanDf.count()

396485

"Description" can be used as the unique indentifer for products in this dataset. We need to convert the text description to integer values, so that we have an index for a vector of products that can be used as inputs and labels for a ML model.

In [6]:
productIndexer = StringIndexer(inputCol="Description", outputCol="ProductIndex") \
.fit(invoicesCleanDf)

timeSeriesDF = productIndexer.transform(invoicesCleanDf) \
.select(["InvoiceDate","InvoiceNo","CustomerID","Description","ProductIndex","Quantity"])
timeSeriesDF.show(3)

+-----------+---------+----------+--------------------+------------+--------+
|InvoiceDate|InvoiceNo|CustomerID|         Description|ProductIndex|Quantity|
+-----------+---------+----------+--------------------+------------+--------+
| 1291191960|   536365|     17850|WHITE HANGING HEA...|         0.0|       6|
| 1291191960|   536365|     17850| WHITE METAL LANTERN|       434.0|       6|
| 1291191960|   536365|     17850|CREAM CUPID HEART...|       452.0|       8|
+-----------+---------+----------+--------------------+------------+--------+
only showing top 3 rows

The time range for the dataset is from December 01, 2010 00:26:00 to December 09, 2011 04:50:00

In [7]:
timeSeriesDF.select("InvoiceDate").agg(F.max("InvoiceDate"),F.min("InvoiceDate")).show()

+----------------+----------------+
|max(InvoiceDate)|min(InvoiceDate)|
+----------------+----------------+
|      1323435000|      1291191960|
+----------------+----------------+

This dataset has 4335 customers

In [8]:
timeSeriesDF.select("CustomerID").distinct().count()

4335

There are a total of 3874 products

In [9]:
timeSeriesDF.select("ProductIndex").distinct().count()

3874

There are 4335 customers with an order history ranging from 1 to 207. The average number of transactions per customer is 4.26 with a standard deviation of 7.67.

In [10]:
txnPerCustomer = timeSeriesDF.groupBy("CustomerID") \
.agg(F.countDistinct(col("InvoiceDate"),col("InvoiceNo")).alias("TxnPerCustomer")).sort(col("TxnPerCustomer").desc())

txnPerCustomer.agg(F.countDistinct("CustomerID").alias("total customers"),     
                 F.max("TxnPerCustomer").alias("max"),
                 F.min("TxnPerCustomer").alias("min"),
                 F.avg("TxnPerCustomer").alias("avg"),
                 F.stddev("TxnPerCustomer").alias("std")).show()

+---------------+---+---+------------------+-----------------+
|total customers|max|min|               avg|              std|
+---------------+---+---+------------------+-----------------+
|           4335|207|  1|4.2551326412918105|7.668065002405116|
+---------------+---+---+------------------+-----------------+

I chose to trim the dataset, so we only consider customers with some minimum amount of order history. I also chose to cap the maximum number of orders, so outliers are removed. RNNs with long sequences suffer from the <a href="https://en.wikipedia.org/wiki/Vanishing_gradient_problem"> vanishing gradient problem </a>. Keeping outliers in the dataset isn't worthwhile. The maximum and minimum of 50 and 5 was selected with little testing. I leave it up to the reader to search for more optimial parameters if desired.

We are now left with 1090 customers with order history between 50 and 5. The average number of transactions within this group of customers is about 10 with a standard deviation of 6.76.

In [11]:
MAX_TXNS = 50
MIN_TXNS = 5

#transactions per customer statistics
#txnPerCustomer = timeSeriesDF.groupBy("CustomerID") \
#.agg(F.countDistinct(col("InvoiceDate"),col("InvoiceNo")).alias("TxnPerCustomer"))

txnPerCustomer = txnPerCustomer \
.filter(txnPerCustomer.TxnPerCustomer <= MAX_TXNS) \
.filter((txnPerCustomer.TxnPerCustomer >= MIN_TXNS))

customerBasket = txnPerCustomer.select(col("CustomerID").astype("float")).collect()
customerBasket = np.array(customerBasket)
N = customerBasket.shape[0]
customerBasket= customerBasket.reshape(N)

txnStats = txnPerCustomer.agg( \
                    F.countDistinct("CustomerID").alias("total customers"),     
                    F.max("TxnPerCustomer").alias("max"),
                    F.min("TxnPerCustomer").alias("min"),
                    F.avg("TxnPerCustomer").alias("avg"),
                    F.stddev("TxnPerCustomer").alias("std")).show()

+---------------+---+---+----------------+-----------------+
|total customers|max|min|             avg|              std|
+---------------+---+---+----------------+-----------------+
|           1090| 50|  5|9.93302752293578|6.762405598972053|
+---------------+---+---+----------------+-----------------+

Now we filter the dataset to only include the orders from customers who have an order history of 5 to 50. We are left with 223881 orders.

In [12]:
basketTimeSeries = timeSeriesDF \
.filter(timeSeriesDF.CustomerID.isin(*customerBasket) == True)

basketTimeSeries.show(5)
basketTimeSeries.count()

+-----------+---------+----------+--------------------+------------+--------+
|InvoiceDate|InvoiceNo|CustomerID|         Description|ProductIndex|Quantity|
+-----------+---------+----------+--------------------+------------+--------+
| 1291191960|   536365|     17850|WHITE HANGING HEA...|         0.0|       6|
| 1291191960|   536365|     17850| WHITE METAL LANTERN|       434.0|       6|
| 1291191960|   536365|     17850|CREAM CUPID HEART...|       452.0|       8|
| 1291191960|   536365|     17850|KNITTED UNION FLA...|       276.0|       6|
| 1291191960|   536365|     17850|RED WOOLLY HOTTIE...|       268.0|       6|
+-----------+---------+----------+--------------------+------------+--------+
only showing top 5 rows

223881

The products need to be re-index after filtering the dataset.

In [13]:
productReIndexer = StringIndexer(inputCol="ProductIndex", outputCol="NewIndex") \
.fit(basketTimeSeries)

basketTimeSeriesReIndexed = productReIndexer.transform(basketTimeSeries) \
.select(["InvoiceDate","InvoiceNo","CustomerID","Description","ProductIndex","NewIndex","Quantity"])
basketTimeSeriesReIndexed.show(3)
basketTimeSeriesReIndexed.count()

+-----------+---------+----------+--------------------+------------+--------+--------+
|InvoiceDate|InvoiceNo|CustomerID|         Description|ProductIndex|NewIndex|Quantity|
+-----------+---------+----------+--------------------+------------+--------+--------+
| 1291191960|   536365|     17850|WHITE HANGING HEA...|         0.0|     0.0|       6|
| 1291191960|   536365|     17850| WHITE METAL LANTERN|       434.0|   391.0|       6|
| 1291191960|   536365|     17850|CREAM CUPID HEART...|       452.0|   344.0|       8|
+-----------+---------+----------+--------------------+------------+--------+--------+
only showing top 3 rows

223881

The most popular product by quantity bought is product 138: "WORLD WAR 2 GLIDERS ASSTD DESIGNS." This is a useful datapoint that is used later when evaluating our predictive model.

In [14]:
basketTimeSeriesReIndexed.select("ProductIndex","Description","Quantity") \
.groupBy("ProductIndex","Description").agg(F.sum(col("Quantity")).alias("TotalQty")) \
.orderBy(col("TotalQty").desc()).limit(10).show(truncate=False)

+------------+----------------------------------+--------+
|ProductIndex|Description                       |TotalQty|
+------------+----------------------------------+--------+
|138.0       |WORLD WAR 2 GLIDERS ASSTD DESIGNS |36890   |
|2.0         |JUMBO BAG RED RETROSPOT           |29850   |
|63.0        |POPCORN HOLDER                    |26280   |
|3.0         |ASSORTED COLOUR BIRD ORNAMENT     |24673   |
|186.0       |PACK OF 12 LONDON TISSUES         |23371   |
|0.0         |WHITE HANGING HEART T-LIGHT HOLDER|22667   |
|8.0         |PACK OF 72 RETROSPOT CAKE CASES   |19052   |
|56.0        |PACK OF 60 PINK PAISLEY CAKE CASES|17889   |
|279.0       |MINI PAINT SET VINTAGE            |17291   |
|30.0        |VICTORIAN GLASS HANGING T-LIGHT   |16588   |
+------------+----------------------------------+--------+

The most popular product by times bought is product 0: "WHITE HANGING HEART T-LIGHT HOLDER." This is a useful datapoint that is used later when evaluating our predictive model.

In [15]:
basketTimeSeriesReIndexed.select("ProductIndex","Description","Quantity") \
.groupBy("ProductIndex","Description") \
.agg(F.count(lit(1)).alias("TimesBought")) \
.orderBy(col("TimesBought").desc()).limit(10).show(truncate=False)

+------------+----------------------------------+-----------+
|ProductIndex|Description                       |TimesBought|
+------------+----------------------------------+-----------+
|0.0         |WHITE HANGING HEART T-LIGHT HOLDER|1227       |
|2.0         |JUMBO BAG RED RETROSPOT           |1054       |
|1.0         |REGENCY CAKESTAND 3 TIER          |995        |
|5.0         |LUNCH BAG RED RETROSPOT           |924        |
|4.0         |PARTY BUNTING                     |852        |
|3.0         |ASSORTED COLOUR BIRD ORNAMENT     |825        |
|7.0         |LUNCH BAG  BLACK SKULL.           |743        |
|6.0         |SET OF 3 CAKE TINS PANTRY DESIGN  |690        |
|11.0        |LUNCH BAG SPACEBOY DESIGN         |646        |
|17.0        |LUNCH BAG SUKI DESIGN             |640        |
+------------+----------------------------------+-----------+

The number of products bought per order ranges from 1 to 540 within our reduced dataset. On average, 20 products are bought with a standard deviation of 24.5. 

In [16]:
MAX_PRODUCTS = 600
productsPerTxn = basketTimeSeriesReIndexed \
.groupBy("InvoiceDate", "CustomerID", "InvoiceNo") \
.agg(F.countDistinct(col("NewIndex")).alias("ProductsPerTxn")) \
.orderBy(col("CustomerID").desc(), col("InvoiceNo").desc(), col("InvoiceDate").desc())

productsPerTxn= productsPerTxn.filter(productsPerTxn.ProductsPerTxn <= MAX_PRODUCTS)

productStats = productsPerTxn.agg( \
    F.countDistinct("InvoiceNo").alias("total txns"),
    F.max("ProductsPerTxn").alias("max"),
    F.min("ProductsPerTxn").alias("min"),
    F.avg("ProductsPerTxn").alias("avg"),
    F.stddev("ProductsPerTxn").alias("std")).show()

+----------+---+---+-----------------+------------------+
|total txns|max|min|              avg|               std|
+----------+---+---+-----------------+------------------+
|     10807|540|  1|20.24032511314307|24.492032572420484|
+----------+---+---+-----------------+------------------+

The following will serve as a product name lookup table for the remaining products in our filtered dataset. 3648 products remain.

In [17]:
releventProducts = basketTimeSeriesReIndexed \
.select("Description", col("NewIndex").alias("Index")).distinct() \
.sort(col("Index").asc())

releventProducts.show()
releventProducts.count()

+--------------------+-----+
|         Description|Index|
+--------------------+-----+
|WHITE HANGING HEA...|  0.0|
|JUMBO BAG RED RET...|  1.0|
|REGENCY CAKESTAND...|  2.0|
|LUNCH BAG RED RET...|  3.0|
|       PARTY BUNTING|  4.0|
|ASSORTED COLOUR B...|  5.0|
|LUNCH BAG  BLACK ...|  6.0|
|SET OF 3 CAKE TIN...|  7.0|
|LUNCH BAG SPACEBO...|  8.0|
|LUNCH BAG SUKI DE...|  9.0|
|LUNCH BAG PINK PO...| 10.0|
|PACK OF 72 RETROS...| 11.0|
|ALARM CLOCK BAKEL...| 12.0|
| LUNCH BAG CARS BLUE| 13.0|
|      SPOTTY BUNTING| 14.0|
|WOODEN PICTURE FR...| 15.0|
|LUNCH BAG APPLE D...| 16.0|
|JUMBO BAG PINK PO...| 17.0|
|ALARM CLOCK BAKEL...| 18.0|
|WOODEN FRAME ANTI...| 19.0|
+--------------------+-----+
only showing top 20 rows

3648

Let's persist this lookup table for use later. Set S3_TARGET_PREFIX to where you want to write out the lookup table. 

In [18]:
S3_TARGET_PREFIX = "/processed/ecomm/customer_basket_list.csv.gz"
S3_LOCATION = "s3://"+S3_BUCKET+S3_TARGET_PREFIX

print(S3_LOCATION)

s3://dtong-ml-datasets/processed/ecomm/customer_basket_list.csv.gz

After the file is written to the S3 location you have provided, you may want to rename the file. Spark will write out the file into a folder with the name of the provided prefix. The actual file will be named by Spark in the form part-xxxx-xxxxxxx.

In [19]:
releventProducts.coalesce(1).write.option("compression","gzip") \
.csv(S3_LOCATION)

'path s3://dtong-ml-datasets/processed/ecomm/customer_basket_list.csv.gz already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 766, in csv
    self._jwrite.csv(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://dtong-ml-datasets/processed/ecomm/customer_basket_list.csv.gz already exists.;'



Below is an intermediate process step. our goal is to prepare a dataset for a RNN where we have a sequence of orders for each customer as input and targets. This intermediate step involves rolling up all the order line items into orders where we have a "cart" column that contains a dense vector of product index values. This vector stores the information about the products that were purchased in the order.

In [20]:
carts= basketTimeSeriesReIndexed \
.groupBy("InvoiceDate", "CustomerID", "InvoiceNo") \
.agg(F.collect_set("NewIndex").alias("Cart")) \
.orderBy(col("CustomerID").desc(),col("InvoiceDate").asc(), col("InvoiceNo").desc()) \
.select("CustomerID","InvoiceDate","InvoiceNo","Cart")

carts.show()
carts.count()

+----------+-----------+---------+--------------------+
|CustomerID|InvoiceDate|InvoiceNo|                Cart|
+----------+-----------+---------+--------------------+
|     18283| 1294323240|   540350|[238.0, 703.0, 6....|
|     18283| 1295794680|   541854|[238.0, 1193.0, 1...|
|     18283| 1298889000|   545079|[0.0, 261.0, 1213...|
|     18283| 1303403820|   550957|[238.0, 621.0, 34...|
|     18283| 1306150380|   554157|[1570.0, 6.0, 10....|
|     18283| 1308051660|   556731|[0.0, 1455.0, 130...|
|     18283| 1308856800|   557956|[0.0, 293.0, 6.0,...|
|     18283| 1310648400|   560025|[0.0, 621.0, 127....|
|     18283| 1310649600|   560032|             [444.0]|
|     18283| 1315226100|   565579|[1147.0, 392.0, 1...|
|     18283| 1319726280|   573093|[293.0, 1537.0, 2...|
|     18283| 1320937140|   575668|[293.0, 306.0, 66...|
|     18283| 1320937620|   575675|             [290.0]|
|     18283| 1322054820|   578262|[238.0, 1537.0, 6...|
|     18283| 1322657940|   579673|[662.0, 318.0,

RNNs requires a fixed size input per timeslice, so we need to convert the dense vectors above to a sparse vector. The sparse vector's length is equal to the number of products, and is made up of zeros and ones. This allows us to use each position in the vector to represent whether a product was bought in the order. For instance, a one in position N means that the product with index value N was bought in the order. Zero indicates the product wasn't bought.

This isn't an efficient representation of the data, and could be problematic for very large product catalogs. In such a situation, implementing an autoencoder maybe worthwhile. In this case, we have 3648 products, and is manageable. 

In [21]:
nProducts= releventProducts.count()

def encodeCart(cart) :
    encoding = ["0"]*nProducts
    
    for idx in cart :     
        encoding[int(idx)] = "1"
  
    return encoding

cartsSparseVecs = carts.rdd.map(lambda r: \
    Row(InvoiceNo=r["InvoiceNo"], InvoiceDate=r["InvoiceDate"], \
        CustomerID=r["CustomerID"], Cart=encodeCart(r["Cart"]))) \
.toDF().select("CustomerID","InvoiceNo","InvoiceDate","Cart")
cartsSparseVecs.show(5,truncate=100)

+----------+---------+-----------+----------------------------------------------------------------------------------------------------+
|CustomerID|InvoiceNo|InvoiceDate|                                                                                                Cart|
+----------+---------+-----------+----------------------------------------------------------------------------------------------------+
|     18283|   540350| 1294323240|[0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, ...|
|     18283|   541854| 1295794680|[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...|
|     18283|   545079| 1298889000|[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...|
|     18283|   550957| 1303403820|[0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, ...|
|     18283|   554157| 1306150380|[0, 0, 0, 1, 0

Let's write out this processed dataset. Update S3_TARGET_PREFIX to the location where you want to write out this dataset.

In [22]:
S3_TARGET_PREFIX = "/processed/ecomm/cart_sparse_vecs.parquet.gz"
S3_LOCATION = "s3://"+S3_BUCKET+S3_TARGET_PREFIX

print(S3_LOCATION)

s3://dtong-ml-datasets/processed/ecomm/cart_sparse_vecs.parquet.gz

Write out the dataset in parquet format. Again, you probably want to rename the file manually afterwards.

In [23]:
cartsSparseVecs.coalesce(1).write.option("compression","gzip") \
.parquet("s3://dtong-ml-datasets/processed/ecomm/cart_sparse_vecs.parquet.gz")

'path s3://dtong-ml-datasets/processed/ecomm/cart_sparse_vecs.parquet.gz already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 691, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://dtong-ml-datasets/processed/ecomm/cart_sparse_vecs.parquet.gz already exists.;'

